# Homework 3 (Convolutional Neural Networks)

Choose a dataset that you're interested in from among these options (or choose your own data set as long as it's large enough and **you check with me** in advance to make sure it'll work):

- [Grape Disease Detection Data](https://www.kaggle.com/datasets/rm1000/augmented-grape-disease-detection-dataset)
- [Indian Bird Data](https://www.kaggle.com/datasets/arjunbasandrai/25-indian-bird-species-with-226k-images)
- [Skin Cancer Classification](https://www.kaggle.com/datasets/kylegraupe/skin-cancer-binary-classification-dataset)
- [Fruit and Veg Detection Data](https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition)
- [Large Scale Fish Data](https://www.kaggle.com/datasets/crowww/a-large-scale-fish-dataset)
- [Berkeley Segmentation Data](https://www.kaggle.com/datasets/balraj98/berkeley-segmentation-dataset-500-bsds500)

Then Build a Deep CONVOLUTIONAL Neural Network (No Recurrent Layers, no Transfer Learning unless approved by Dr. P ahead of time, no Generative Models) using keras/tensorflow (at least 5 Convolutional Layers, and at least 3 Pooling Layers) to do one of the following tasks:

- Classify Images (e.g. Hot Dog vs. Not a Hot Dog)
- Compress Images (e.g. with a Denoising Convolutional AutoEncoder)
- Detect/Segment Objects (e.g. what pixels in the image contain a cat?)

Make sure that:

- your NN has some sort of regularization (or multiple types if needed)
- you've properly formatted and inputted your data into the network
- your model architechture and loss function are appropriate for the problem
- you print out at least 2 metrics for both train and test data to examine


Then create a **technical report** discussing your model building process, the results, and your reflection on it. The report should follow the format in the example including an Introduction, Analysis, Methods, Results, and Reflection section.

# Introduction
An introduction should introduce the problem you're working on, give some background and relevant detail for the reader, and explain why it is important. 

# Analysis 
Any exploratory analysis of your data, and general summarization of the data (e.g. summary statistics, correlation heatmaps, graphs, information about the data...). This can also include any cleaning and joining you did. 

# Methods
Explain the structure of your model and your approach to building it. This can also include changes you made to your model in the process of building it. Someone should be able to read your methods section and *generally* be able to tell exactly what architechture you used. 

# Results
Detailed discussion of how your model performed, and your discussion of how your model performed.

# Reflection
Reflections on what you learned/discovered in the process of doing the assignment. Things you would do differently in the future, ways you'll approach similar problems in the future, etc.


# What to Turn In

- PDF of your technical report
- your code as a .py, .ipynb, or link to github (you must turn it in either as a file, or a link to something that has timestamps of when the file was last edited)
- a README file as a .txt or .md

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras as kb
from tensorflow.keras import backend